The source of this notebook: https://github.com/Azure-Samples/MachineLearningSamples-DeepLearningforPredictiveMaintenance

# Step 3: Model Operationalization & Deployment

In the `Code\2_model_building_and_evaluation.ipynb` notebook, we trained an LSTM neural net to predict aircraft engine failure 30 cycles into the future.

In this notebook, we will create the artifacts and scripts to deploy the LSTM model into a webservice on Azure. The artifacts include the model files, and test scripts to validate your model can be used to predict future reliability of the engines based on the present operating characteristics.

In [1]:
print('Importing libraries is in progress...')

import tensorflow as tf
import keras
from keras.models import model_from_json
import os
import sys # It is needed to do pip operations in Python script.
#!{sys.executable} -m pip install sklearn
#!{sys.executable} -m pip install enum34
#!{sys.executable} -m pip -y uninstall pandas
#!{sys.executable} -m pip install pandas==1.4.1 # Because of a bug Pandas 1.12 needed.
import pandas as pd
#print(pd.__version__) 
from pandas import read_csv
from pandas import DataFrame
#!{sys.executable} -m pip install azure-ml-api-sdk # It is needed to use azureml.api modul.
#!{sys.executable} -m pip show azure-ml-api-sdk
import numpy as np
from numpy import concatenate
import json
import shutil
import h5py
import urllib
import requests
from sklearn.preprocessing import MinMaxScaler

import azureml
import azureml.core
from azureml.core import Workspace
from azureml.core import Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice

# For creating the deployment schema file
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema

print('Importing libraries completed successfully.')

Importing libraries is in progress...
Importing libraries completed successfully.


## Load the test data frame

In [ ]:
test_df = pd.read_csv('./data/preparedData/PM_test_prepared.csv', sep=",", header=0)
test_df.head(10)

We will need to recreate the feature engineering (creating the sequence features) just as we did in the model building step. We will do this within the webservice so that the service can take the raw sensor data, and return a scored result predicting probability of failure at 30 days (`label1`). 

When scoring an unseen observation, the model will not know the true labels. Therefore, we create a `score_df` without labels.

In [ ]:
print('Column operations are in progress...')

# pick the feature columns 
# Sequence help order the observations in "time"
sequence_cols = ['cycle_norm']

# key columns group the machines
key_cols = ['machineID', 'cycle']

# Labels are what we're predicting.
label_cols = ['label1', 'label2', 'RUL']

# The scoreing data should not have labels... if we knew the label, 
# we wouldn'y need to predict.
score_df = test_df.drop(label_cols, axis = 1)

print('Column operations completed successfully.')

### Test init() and run() functions

The web service requires two functions, an `init()` function that will initialize the web service by loading the model into the service, and a `run()` function that will engineer the features to match the model call structure, and score that data set. We create the functions in here for testing and debugging.

In [ ]:
# Register the model
model = Model.register(ws, model_name="machineRULPredictor_model", model_path="./models/machineRULPredictor.h5")
print('Model registered.')

In [ ]:
print('Function init() defining is in progress...')

def init():
    # read in the model file
    #from keras.models import model_from_json
    global model
    
    # load json and create model
    #with open('./models/machineRULPredictor.json', 'r') as json_file:
    #    loaded_model_json = json_file.read()
    #    json_file.close()
    #    model = model_from_json(loaded_model_json)
    
    # load weights into new model
    #model.load_weights('./models/machineRULPredictor_weights.h5')
    #model.compile('sgd','mse',run_eagerly=True)
    
    model = keras.models.load_model('./models/machineRULPredictor.h5')

print('Function init() defining completed successfully.')

In [ ]:
print('Function run() defining is in progress...')

def run(score_input): 
    # Create the sequences
    sequence_length = 137
    sequence_cols = ['cycle_norm']
    key_cols = ['machineID', 'cycle']

    # Feature engineering
    input_features = score_input.columns.values.tolist()
    sensor_cols = [x for x in input_features if x not in set(key_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    sequence_cols.extend(sensor_cols)
    
    seq_array = [score_input[score_input['machineID']==id][sequence_cols].values[-sequence_length:] 
                 for id in score_input['machineID'].unique() if len(score_input[score_input['machineID']==id]) >= sequence_length]

    seq_array = np.asarray(seq_array).astype(np.float32)
    # There are some problems with the data format so I try with hard coded data:
    seq_array = np.asarray([[[0.14923596, 0.66666667, 0.83333333, 0.54959738, 0.71285325,
        0.4659503 , 0.59001228],
       [0.14943441, 0.66666667, 0.83333333, 0.32982956, 0.44686331,
        0.34200524, 0.32326547],
       [0.14963286, 0.66666667, 0.83333333, 0.53332076, 0.64905333,
        0.59418519, 0.36811938],
       [0.14983132, 0.66666667, 0.83333333, 0.5451522 , 0.76276414,
        0.45844232, 0.43014104],
       [0.15002977, 0.66666667, 0.83333333, 0.5194982 , 0.6271168 ,
        0.43638889, 0.56395535],
       [0.15022822, 0.66666667, 0.83333333, 0.52537654, 0.51916574,
        0.52431215, 0.42782451],
       [0.15042667, 0.66666667, 0.83333333, 0.32904783, 0.64517548,
        0.40470976, 0.55790012],
       [0.15062512, 0.66666667, 0.83333333, 0.43717643, 0.49219885,
        0.45868012, 0.30046295],
       [0.15082358, 0.66666667, 0.83333333, 0.36671982, 0.55367248,
        0.40065109, 0.52886584],
       [0.15102203, 0.66666667, 0.83333333, 0.51124049, 0.63112986,
        0.25788691, 0.47391496],
       [0.15122048, 0.66666667, 0.83333333, 0.2417658 , 0.59180895,
        0.57995931, 0.48146367],
       [0.15141893, 0.66666667, 0.83333333, 0.36933083, 0.51539482,
        0.3783349 , 0.27235855],
       [0.15161738, 0.66666667, 0.83333333, 0.28297409, 0.81124665,
        0.45256192, 0.25947621],
       [0.15181584, 0.66666667, 0.83333333, 0.4457952 , 0.69949822,
        0.25438028, 0.44934334],
       [0.15201429, 0.66666667, 0.83333333, 0.42523873, 0.80604309,
        0.25983895, 0.54019328],
       [0.15221274, 0.66666667, 0.83333333, 0.63415984, 0.3699069 ,
        0.4944037 , 0.38384921],
       [0.15241119, 0.66666667, 0.83333333, 0.4551023 , 0.5304087 ,
        0.3837343 , 0.43567077],
       [0.15260964, 0.66666667, 0.83333333, 0.4726882 , 0.54275567,
        0.24722907, 0.44742661],
       [0.1528081 , 0.66666667, 0.83333333, 0.29381008, 0.61726852,
        0.36389851, 0.52417436],
       [0.15300655, 0.66666667, 0.83333333, 0.46860201, 0.57928542,
        0.35390927, 0.68470181],
       [0.153205  , 0.66666667, 0.83333333, 0.57945427, 0.6376952 ,
        0.44746134, 0.53037403],
       [0.15340345, 0.66666667, 0.83333333, 0.41114418, 0.35172953,
        0.37474705, 0.29365747],
       [0.15360191, 0.66666667, 0.83333333, 0.63291764, 0.60836935,
        0.32145429, 0.42138587],
       [0.15380036, 0.66666667, 0.83333333, 0.64736869, 0.52710838,
        0.57118695, 0.38133895],
       [0.15399881, 0.66666667, 0.83333333, 0.44512835, 0.60985738,
        0.27156206, 0.45869594],
       [0.15419726, 0.66666667, 0.83333333, 0.51941227, 0.50531686,
        0.36836507, 0.41389907],
       [0.15439571, 0.66666667, 0.83333333, 0.53364621, 0.58818619,
        0.4316318 , 0.43980045],
       [0.15459417, 0.66666667, 0.83333333, 0.51970799, 0.56567368,
        0.47194062, 0.55323922],
       [0.15479262, 0.66666667, 0.83333333, 0.47126073, 0.47784115,
        0.39066287, 0.48839372],
       [0.15499107, 0.66666667, 0.83333333, 0.45879882, 0.41754077,
        0.42164666, 0.59105837],
       [0.15518952, 0.66666667, 0.83333333, 0.63909896, 0.60294843,
        0.3628267 , 0.50384362],
       [0.15538797, 0.66666667, 0.83333333, 0.36354546, 0.53744577,
        0.44103122, 0.67369821],
       [0.15558643, 0.66666667, 0.83333333, 0.58069986, 0.67301672,
        0.62658603, 0.44383956],
       [0.15578488, 0.66666667, 0.83333333, 0.51963754, 0.62165717,
        0.44367806, 0.56101512],
       [0.15598333, 0.66666667, 0.83333333, 0.5715269 , 0.63375637,
        0.52343773, 0.4628571 ],
       [0.15618178, 0.66666667, 0.83333333, 0.38640497, 0.56249479,
        0.23782059, 0.40762717],
       [0.15638023, 0.66666667, 0.83333333, 0.33367478, 0.47152284,
        0.41011438, 0.59308901],
       [0.15657869, 0.66666667, 0.83333333, 0.51220981, 0.47578146,
        0.52260533, 0.47412645],
       [0.15677714, 0.66666667, 0.83333333, 0.77109782, 0.46679228,
        0.37795948, 0.42412404],
       [0.15697559, 0.66666667, 0.83333333, 0.50093316, 0.47887974,
        0.36843522, 0.61027864],
       [0.15717404, 0.66666667, 0.83333333, 0.47018212, 0.61380245,
        0.40213331, 0.46886895],
       [0.15737249, 0.66666667, 0.83333333, 0.56340243, 0.5532862 ,
        0.38094165, 0.41965739],
       [0.15757095, 0.66666667, 0.83333333, 0.43051235, 0.67650223,
        0.3580468 , 0.45080217],
       [0.1577694 , 0.66666667, 0.83333333, 0.47665373, 0.48382433,
        0.53466701, 0.1589085 ],
       [0.15796785, 0.66666667, 0.83333333, 0.3260423 , 0.51593496,
        0.58094886, 0.55054216],
       [0.1581663 , 0.66666667, 0.83333333, 0.60309922, 0.35894008,
        0.5838989 , 0.44449311],
       [0.15836475, 0.66666667, 0.83333333, 0.5162941 , 0.40081931,
        0.48132487, 0.51347364],
       [0.15856321, 0.66666667, 0.83333333, 0.45709945, 0.49998431,
        0.4605301 , 0.50932976],
       [0.15876166, 0.66666667, 0.83333333, 0.51786671, 0.67431214,
        0.46701809, 0.55315776],
       [0.15896011, 0.66666667, 0.83333333, 0.48397312, 0.48989347,
        0.35261583, 0.38139004],
       [0.15915856, 0.66666667, 0.83333333, 0.42332106, 0.69158878,
        0.34590006, 0.33949373],
       [0.15935702, 0.66666667, 0.83333333, 0.55448873, 0.57671654,
        0.3516092 , 0.55213481],
       [0.15955547, 0.66666667, 0.83333333, 0.48812941, 0.80699254,
        0.38622158, 0.47501884],
       [0.15975392, 0.66666667, 0.83333333, 0.43160286, 0.56332529,
        0.41736253, 0.28903521],
       [0.15995237, 0.66666667, 0.83333333, 0.46535446, 0.58717833,
        0.45972087, 0.58613506],
       [0.16015082, 0.66666667, 0.83333333, 0.74726503, 0.49276759,
        0.41813327, 0.47798332],
       [0.16034928, 0.66666667, 0.83333333, 0.61418334, 0.65741431,
        0.38458203, 0.58876552],
       [0.16054773, 0.66666667, 0.83333333, 0.39066175, 0.6251094 ,
        0.330608  , 0.44976676],
       [0.16074618, 0.66666667, 0.83333333, 0.57586869, 0.5665388 ,
        0.33425085, 0.48587755],
       [0.16094463, 0.66666667, 0.83333333, 0.55025925, 0.62534008,
        0.44671277, 0.44825944],
       [0.16114308, 0.66666667, 0.83333333, 0.48269987, 0.54751105,
        0.49994113, 0.48505293],
       [0.16134154, 0.66666667, 0.83333333, 0.41594811, 0.6556298 ,
        0.41589057, 0.45633174],
       [0.16153999, 0.66666667, 0.83333333, 0.4428962 , 0.73697263,
        0.3571785 , 0.36217655],
       [0.16173844, 0.66666667, 0.83333333, 0.53464726, 0.62812843,
        0.39391375, 0.50010913],
       [0.16193689, 0.66666667, 0.83333333, 0.42007096, 0.80603049,
        0.55924483, 0.41654737],
       [0.16213534, 0.66666667, 0.83333333, 0.57862197, 0.58894857,
        0.49632225, 0.5117699 ],
       [0.1623338 , 0.66666667, 0.83333333, 0.50416792, 0.50804178,
        0.30764597, 0.51120034],
       [0.16253225, 0.66666667, 0.83333333, 0.50503752, 0.46017328,
        0.3385565 , 0.34355097],
       [0.1627307 , 0.66666667, 0.83333333, 0.41954427, 0.63592578,
        0.6314654 , 0.38704159],
       [0.16292915, 0.66666667, 0.83333333, 0.38463039, 0.60973922,
        0.48721192, 0.39881377],
       [0.1631276 , 0.66666667, 0.83333333, 0.549986  , 0.71281546,
        0.51182148, 0.56206297],
       [0.16332606, 0.66666667, 0.83333333, 0.38202152, 0.50785138,
        0.38198331, 0.51334509],
       [0.16352451, 0.66666667, 0.83333333, 0.5164104 , 0.27769907,
        0.37643472, 0.48655834],
       [0.16372296, 0.66666667, 0.83333333, 0.57574567, 0.46440594,
        0.30503373, 0.51972874],
       [0.16392141, 0.66666667, 0.83333333, 0.44032988, 0.66568158,
        0.38604651, 0.49465378],
       [0.16411987, 0.66666667, 0.83333333, 0.50992369, 0.633003  ,
        0.33140473, 0.49447151],
       [0.16431832, 0.66666667, 0.83333333, 0.50761729, 0.61803431,
        0.62141008, 0.57884557],
       [0.16451677, 0.66666667, 0.83333333, 0.46913516, 0.46799589,
        0.43305629, 0.46683875],
       [0.16471522, 0.66666667, 0.83333333, 0.43184296, 0.71846233,
        0.36597951, 0.5033579 ],
       [0.16491367, 0.66666667, 0.83333333, 0.37770263, 0.72043274,
        0.44064234, 0.47744734],
       [0.16511213, 0.66666667, 0.83333333, 0.58917584, 0.61756397,
        0.38988345, 0.29738863],
       [0.16531058, 0.66666667, 0.83333333, 0.66306578, 0.54698481,
        0.29816112, 0.44046069],
       [0.16550903, 0.66666667, 0.83333333, 0.34753011, 0.62429571,
        0.42018128, 0.54422186],
       [0.16570748, 0.66666667, 0.83333333, 0.42084169, 0.59899732,
        0.34538273, 0.68062549],
       [0.16590593, 0.66666667, 0.83333333, 0.53714766, 0.5233176 ,
        0.31867285, 0.51455606],
       [0.16610439, 0.66666667, 0.83333333, 0.46459716, 0.79728447,
        0.29287657, 0.54589608],
       [0.16630284, 0.66666667, 0.83333333, 0.60685885, 0.7627859 ,
        0.53102514, 0.36463042],
       [0.16650129, 0.66666667, 0.83333333, 0.45016676, 0.5371357 ,
        0.36234758, 0.48067005],
       [0.16669974, 0.66666667, 0.83333333, 0.32630458, 0.6502717 ,
        0.29977745, 0.31127834],
       [0.16689819, 0.66666667, 0.83333333, 0.61934091, 0.68324217,
        0.43857282, 0.40336865],
       [0.16709665, 0.66666667, 0.83333333, 0.53289786, 0.64045468,
        0.4707726 , 0.33293981],
       [0.1672951 , 0.66666667, 0.83333333, 0.62172647, 0.80239406,
        0.24065814, 0.45971052],
       [0.16749355, 0.66666667, 0.83333333, 0.56874513, 0.55327686,
        0.30763078, 0.42705133],
       [0.167692  , 0.66666667, 0.83333333, 0.75409067, 0.70157827,
        0.33767338, 0.40823682],
       [0.16789045, 0.66666667, 0.83333333, 0.55806708, 0.59841169,
        0.42723556, 0.46425731],
       [0.16808891, 0.66666667, 0.83333333, 0.69033112, 0.650277  ,
        0.57144895, 0.3518286 ],
       [0.16828736, 0.66666667, 0.83333333, 0.53220468, 0.77556955,
        0.54382698, 0.49633586],
       [0.16848581, 0.66666667, 0.83333333, 0.52509087, 0.41871796,
        0.53415646, 0.3309969 ],
       [0.16868426, 0.66666667, 0.83333333, 0.6630777 , 0.36348843,
        0.21869866, 0.42808497],
       [0.16888271, 0.66666667, 0.83333333, 0.66012703, 0.50377388,
        0.46911669, 0.42108926],
       [0.16908117, 0.66666667, 0.83333333, 0.65719551, 0.36174508,
        0.30243667, 0.66768381],
       [0.16927962, 0.66666667, 0.83333333, 0.62825936, 0.57472729,
        0.38454321, 0.64870015],
       [0.16947807, 0.66666667, 0.83333333, 0.61402683, 0.81333739,
        0.35459519, 0.65223118],
       [0.16967652, 0.66666667, 0.83333333, 0.64476372, 0.61324651,
        0.3916903 , 0.3364507 ],
       [0.16987498, 0.66666667, 0.83333333, 0.71781102, 0.65292058,
        0.24742094, 0.57611908],
       [0.17007343, 0.66666667, 0.83333333, 0.5376592 , 0.78369732,
        0.42405026, 0.6207278 ],
       [0.17027188, 0.66666667, 0.83333333, 0.58864504, 0.65972945,
        0.36872454, 0.51423921],
       [0.17047033, 0.66666667, 0.83333333, 0.45362191, 0.68462962,
        0.53711181, 0.36674166],
       [0.17066878, 0.66666667, 0.83333333, 0.53549992, 0.49845023,
        0.21808856, 0.4229779 ],
       [0.17086724, 0.66666667, 0.83333333, 0.56127937, 0.67585306,
        0.29947504, 0.35929855],
       [0.17106569, 0.66666667, 0.83333333, 0.58591761, 0.61611436,
        0.35467961, 0.45090625],
       [0.17126414, 0.66666667, 0.83333333, 0.64491204, 0.69148971,
        0.20742524, 0.48409945],
       [0.17146259, 0.66666667, 0.83333333, 0.85195323, 0.62527227,
        0.35117042, 0.25665912],
       [0.17166104, 0.66666667, 0.83333333, 0.57876465, 0.5611623 ,
        0.4816895 , 0.37922238],
       [0.1718595 , 0.66666667, 0.83333333, 0.48242571, 0.56956496,
        0.29905694, 0.29823658],
       [0.17205795, 0.66666667, 0.83333333, 0.54745425, 0.61160209,
        0.27894404, 0.48696811],
       [0.1722564 , 0.66666667, 0.83333333, 0.62351399, 0.6417171 ,
        0.51692447, 0.48082924],
       [0.17245485, 0.66666667, 0.83333333, 0.67285432, 0.76966338,
        0.44123265, 0.44093442],
       [0.1726533 , 0.66666667, 0.83333333, 0.49849421, 0.54406418,
        0.32597676, 0.5700005 ],
       [0.17285176, 0.66666667, 0.83333333, 0.8202298 , 0.46685339,
        0.51506072, 0.46946898],
       [0.17305021, 0.66666667, 0.83333333, 0.728321  , 0.69478484,
        0.49997131, 0.4046414 ],
       [0.17324866, 0.66666667, 0.83333333, 0.58467526, 0.88585612,
        0.3456766 , 0.40502593],
       [0.17344711, 0.66666667, 0.83333333, 0.56592784, 0.55894564,
        0.6284823 , 0.50010475],
       [0.17364556, 0.66666667, 0.83333333, 0.61675033, 0.72906911,
        0.35640598, 0.35105949],
       [0.17384402, 0.66666667, 0.83333333, 0.75741633, 0.60916402,
        0.55898595, 0.60063631],
       [0.17404247, 0.66666667, 0.83333333, 0.67919303, 0.35581607,
        0.19058489, 0.35706113],
       [0.17424092, 0.66666667, 0.83333333, 0.73898873, 0.63057041,
        0.50616972, 0.50462219],
       [0.17443937, 0.66666667, 0.83333333, 0.55369508, 0.65903964,
        0.4542438 , 0.37052034],
       [0.17463782, 0.66666667, 0.83333333, 0.53888605, 0.32299718,
        0.49203166, 0.33765294],
       [0.17483628, 0.66666667, 0.83333333, 0.52500914, 0.5056947 ,
        0.2946801 , 0.42697432],
       [0.17503473, 0.66666667, 0.83333333, 0.76213175, 0.57095799,
        0.25609406, 0.57514827],
       [0.17523318, 0.66666667, 0.83333333, 0.77096287, 0.49835092,
        0.22879404, 0.39076878],
       [0.17543163, 0.66666667, 0.83333333, 0.70702404, 0.56699757,
        0.28395286, 0.4250315 ],
       [0.17563009, 0.66666667, 0.83333333, 0.80159978, 0.6493734 ,
        0.60192788, 0.41643542],
       [0.17582854, 0.66666667, 0.83333333, 0.69704148, 0.58654211,
        0.15221453, 0.25923464],
       [0.17602699, 0.66666667, 0.83333333, 0.51044309, 0.58028864,
        0.18375871, 0.3012168 ],
       [0.17622544, 0.66666667, 0.83333333, 0.58040526, 0.89435979,
        0.43123124, 0.44607091]]]).astype(np.float32)
    try:
        prediction = model.predict(seq_array)
        pred = prediction.tolist()
        return(pred)
    except Exception as e:
        return(str(e))

print('Function run() defining completed successfully.')

Check that there are X unique engine IDs. 

In [ ]:
print(score_df.machineID.unique())

The webservice test requires an `initialize` of the webservice, then send the entire scoring data set into the model. We expect to get 1 probability prediction for each engine in the scoring data set. Since the `score_df` has x machines, we expect x probabilities back.

In [ ]:
init()

prb=run(score_df)
print(prb)

If we send a machine with fewer than 'sequence_length' records we get the back error. 

In [ ]:
tst_df=score_df.loc[score_df['machineID'] == 89]

print(tst_df.shape)

# Because 
run(tst_df)

If we send a complete data set, we get a probability back.

In [ ]:
tst_df=score_df.loc[score_df['machineID'] == 100]

print(tst_df.shape)

# Because 
ans=run(tst_df)

print(ans)

## Persist model assets

Next we persist the assets we have created for use in operationalization. First we need to define the schema so the webservice knows what the payload data will look like as it comes in.

In [ ]:
print('Input schema file operations are inf progress...')

# define the input data frame
inputs = {"score_input": SampleDefinition(DataTypes.PANDAS, score_df)}

json_schema = generate_schema(run_func=run, inputs=inputs, filepath='./schema/service_schema.json')

# save the schema file for deployment
out = json.dumps(json_schema)
with open('./schema/service_schema.json', 'w') as f:
    f.write(out)

print('Input schema file operations completed successfully.')

The conda dependencies are defined in this `webservices_conda.yaml` file. This will be used to tell the webservice server which python packages are required to run this web service.

In [ ]:
%%writefile './config/webservices_conda.yaml'

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for managed runs. These include runs against
# the localdocker, remotedocker, and cluster compute targets.

# Note that this file is NOT used to automatically manage dependencies for the
# local compute target. To provision these dependencies locally, run:
# conda env update --file conda_dependencies.yml

# Details about the Conda environment file format:
# https://conda.io/docs/using/envs.html#create-environment-file-by-hand

# For managing Spark packages and configuration, see spark_dependencies.yml.

name: project_environment
channels:
- conda-forge
- defaults
dependencies:
  - python=3.8.0
  - tensorflow #=2.0.0
  - pip:
    - azureml
    - azureml-defaults
    - azure-ml-api-sdk
    - azure-common #==1.1.13
    #- azure-storage #==0.36.0
    - numpy #==1.17.0 
    - sklearn
    - tensorflow #==2.0.0
    - keras
    - h5py

The `lstmscore.py` file is python code defining the web service operation. It includes both the `init()` and `run()` functions defined earlier imports the required libraries. These should be nearly identical to the previous defined versions.

In [ ]:
%%writefile './config/lstmscore.py'

# import the libraries
import tensorflow as tf
import keras
import json
import shutil
import numpy as np
import pandas as pd
from azureml.core import Model

def init():
    # read in the model file
    global model
    
    model_path = Model.get_model_path('machineRULPredictor_model')
    model = tf.keras.models.load_model(model_path)

def run(score_input):
    # Create the sequences
    sequence_length = 137
    sequence_cols = ['cycle_norm']
    key_cols = ['machineID', 'cycle']
    
    # There are some problems with the data format so I try with hard coded data:
    #input_features = score_input.columns.values.tolist()
    #sensor_cols = [x for x in input_features if x not in set(key_cols)]
    #sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    #sequence_cols.extend(sensor_cols)
    
    #seq_array = [score_input[score_input['machineID']==id][sequence_cols].values[-sequence_length:] 
    #             for id in score_input['machineID'].unique() if len(score_input[score_input['machineID']==id]) >= sequence_length]

    #seq_array = np.asarray(seq_array).astype(np.float32)
    seq_array = np.asarray([[[0.14923596, 0.66666667, 0.83333333, 0.54959738, 0.71285325,
        0.4659503 , 0.59001228],
       [0.14943441, 0.66666667, 0.83333333, 0.32982956, 0.44686331,
        0.34200524, 0.32326547],
       [0.14963286, 0.66666667, 0.83333333, 0.53332076, 0.64905333,
        0.59418519, 0.36811938],
       [0.14983132, 0.66666667, 0.83333333, 0.5451522 , 0.76276414,
        0.45844232, 0.43014104],
       [0.15002977, 0.66666667, 0.83333333, 0.5194982 , 0.6271168 ,
        0.43638889, 0.56395535],
       [0.15022822, 0.66666667, 0.83333333, 0.52537654, 0.51916574,
        0.52431215, 0.42782451],
       [0.15042667, 0.66666667, 0.83333333, 0.32904783, 0.64517548,
        0.40470976, 0.55790012],
       [0.15062512, 0.66666667, 0.83333333, 0.43717643, 0.49219885,
        0.45868012, 0.30046295],
       [0.15082358, 0.66666667, 0.83333333, 0.36671982, 0.55367248,
        0.40065109, 0.52886584],
       [0.15102203, 0.66666667, 0.83333333, 0.51124049, 0.63112986,
        0.25788691, 0.47391496],
       [0.15122048, 0.66666667, 0.83333333, 0.2417658 , 0.59180895,
        0.57995931, 0.48146367],
       [0.15141893, 0.66666667, 0.83333333, 0.36933083, 0.51539482,
        0.3783349 , 0.27235855],
       [0.15161738, 0.66666667, 0.83333333, 0.28297409, 0.81124665,
        0.45256192, 0.25947621],
       [0.15181584, 0.66666667, 0.83333333, 0.4457952 , 0.69949822,
        0.25438028, 0.44934334],
       [0.15201429, 0.66666667, 0.83333333, 0.42523873, 0.80604309,
        0.25983895, 0.54019328],
       [0.15221274, 0.66666667, 0.83333333, 0.63415984, 0.3699069 ,
        0.4944037 , 0.38384921],
       [0.15241119, 0.66666667, 0.83333333, 0.4551023 , 0.5304087 ,
        0.3837343 , 0.43567077],
       [0.15260964, 0.66666667, 0.83333333, 0.4726882 , 0.54275567,
        0.24722907, 0.44742661],
       [0.1528081 , 0.66666667, 0.83333333, 0.29381008, 0.61726852,
        0.36389851, 0.52417436],
       [0.15300655, 0.66666667, 0.83333333, 0.46860201, 0.57928542,
        0.35390927, 0.68470181],
       [0.153205  , 0.66666667, 0.83333333, 0.57945427, 0.6376952 ,
        0.44746134, 0.53037403],
       [0.15340345, 0.66666667, 0.83333333, 0.41114418, 0.35172953,
        0.37474705, 0.29365747],
       [0.15360191, 0.66666667, 0.83333333, 0.63291764, 0.60836935,
        0.32145429, 0.42138587],
       [0.15380036, 0.66666667, 0.83333333, 0.64736869, 0.52710838,
        0.57118695, 0.38133895],
       [0.15399881, 0.66666667, 0.83333333, 0.44512835, 0.60985738,
        0.27156206, 0.45869594],
       [0.15419726, 0.66666667, 0.83333333, 0.51941227, 0.50531686,
        0.36836507, 0.41389907],
       [0.15439571, 0.66666667, 0.83333333, 0.53364621, 0.58818619,
        0.4316318 , 0.43980045],
       [0.15459417, 0.66666667, 0.83333333, 0.51970799, 0.56567368,
        0.47194062, 0.55323922],
       [0.15479262, 0.66666667, 0.83333333, 0.47126073, 0.47784115,
        0.39066287, 0.48839372],
       [0.15499107, 0.66666667, 0.83333333, 0.45879882, 0.41754077,
        0.42164666, 0.59105837],
       [0.15518952, 0.66666667, 0.83333333, 0.63909896, 0.60294843,
        0.3628267 , 0.50384362],
       [0.15538797, 0.66666667, 0.83333333, 0.36354546, 0.53744577,
        0.44103122, 0.67369821],
       [0.15558643, 0.66666667, 0.83333333, 0.58069986, 0.67301672,
        0.62658603, 0.44383956],
       [0.15578488, 0.66666667, 0.83333333, 0.51963754, 0.62165717,
        0.44367806, 0.56101512],
       [0.15598333, 0.66666667, 0.83333333, 0.5715269 , 0.63375637,
        0.52343773, 0.4628571 ],
       [0.15618178, 0.66666667, 0.83333333, 0.38640497, 0.56249479,
        0.23782059, 0.40762717],
       [0.15638023, 0.66666667, 0.83333333, 0.33367478, 0.47152284,
        0.41011438, 0.59308901],
       [0.15657869, 0.66666667, 0.83333333, 0.51220981, 0.47578146,
        0.52260533, 0.47412645],
       [0.15677714, 0.66666667, 0.83333333, 0.77109782, 0.46679228,
        0.37795948, 0.42412404],
       [0.15697559, 0.66666667, 0.83333333, 0.50093316, 0.47887974,
        0.36843522, 0.61027864],
       [0.15717404, 0.66666667, 0.83333333, 0.47018212, 0.61380245,
        0.40213331, 0.46886895],
       [0.15737249, 0.66666667, 0.83333333, 0.56340243, 0.5532862 ,
        0.38094165, 0.41965739],
       [0.15757095, 0.66666667, 0.83333333, 0.43051235, 0.67650223,
        0.3580468 , 0.45080217],
       [0.1577694 , 0.66666667, 0.83333333, 0.47665373, 0.48382433,
        0.53466701, 0.1589085 ],
       [0.15796785, 0.66666667, 0.83333333, 0.3260423 , 0.51593496,
        0.58094886, 0.55054216],
       [0.1581663 , 0.66666667, 0.83333333, 0.60309922, 0.35894008,
        0.5838989 , 0.44449311],
       [0.15836475, 0.66666667, 0.83333333, 0.5162941 , 0.40081931,
        0.48132487, 0.51347364],
       [0.15856321, 0.66666667, 0.83333333, 0.45709945, 0.49998431,
        0.4605301 , 0.50932976],
       [0.15876166, 0.66666667, 0.83333333, 0.51786671, 0.67431214,
        0.46701809, 0.55315776],
       [0.15896011, 0.66666667, 0.83333333, 0.48397312, 0.48989347,
        0.35261583, 0.38139004],
       [0.15915856, 0.66666667, 0.83333333, 0.42332106, 0.69158878,
        0.34590006, 0.33949373],
       [0.15935702, 0.66666667, 0.83333333, 0.55448873, 0.57671654,
        0.3516092 , 0.55213481],
       [0.15955547, 0.66666667, 0.83333333, 0.48812941, 0.80699254,
        0.38622158, 0.47501884],
       [0.15975392, 0.66666667, 0.83333333, 0.43160286, 0.56332529,
        0.41736253, 0.28903521],
       [0.15995237, 0.66666667, 0.83333333, 0.46535446, 0.58717833,
        0.45972087, 0.58613506],
       [0.16015082, 0.66666667, 0.83333333, 0.74726503, 0.49276759,
        0.41813327, 0.47798332],
       [0.16034928, 0.66666667, 0.83333333, 0.61418334, 0.65741431,
        0.38458203, 0.58876552],
       [0.16054773, 0.66666667, 0.83333333, 0.39066175, 0.6251094 ,
        0.330608  , 0.44976676],
       [0.16074618, 0.66666667, 0.83333333, 0.57586869, 0.5665388 ,
        0.33425085, 0.48587755],
       [0.16094463, 0.66666667, 0.83333333, 0.55025925, 0.62534008,
        0.44671277, 0.44825944],
       [0.16114308, 0.66666667, 0.83333333, 0.48269987, 0.54751105,
        0.49994113, 0.48505293],
       [0.16134154, 0.66666667, 0.83333333, 0.41594811, 0.6556298 ,
        0.41589057, 0.45633174],
       [0.16153999, 0.66666667, 0.83333333, 0.4428962 , 0.73697263,
        0.3571785 , 0.36217655],
       [0.16173844, 0.66666667, 0.83333333, 0.53464726, 0.62812843,
        0.39391375, 0.50010913],
       [0.16193689, 0.66666667, 0.83333333, 0.42007096, 0.80603049,
        0.55924483, 0.41654737],
       [0.16213534, 0.66666667, 0.83333333, 0.57862197, 0.58894857,
        0.49632225, 0.5117699 ],
       [0.1623338 , 0.66666667, 0.83333333, 0.50416792, 0.50804178,
        0.30764597, 0.51120034],
       [0.16253225, 0.66666667, 0.83333333, 0.50503752, 0.46017328,
        0.3385565 , 0.34355097],
       [0.1627307 , 0.66666667, 0.83333333, 0.41954427, 0.63592578,
        0.6314654 , 0.38704159],
       [0.16292915, 0.66666667, 0.83333333, 0.38463039, 0.60973922,
        0.48721192, 0.39881377],
       [0.1631276 , 0.66666667, 0.83333333, 0.549986  , 0.71281546,
        0.51182148, 0.56206297],
       [0.16332606, 0.66666667, 0.83333333, 0.38202152, 0.50785138,
        0.38198331, 0.51334509],
       [0.16352451, 0.66666667, 0.83333333, 0.5164104 , 0.27769907,
        0.37643472, 0.48655834],
       [0.16372296, 0.66666667, 0.83333333, 0.57574567, 0.46440594,
        0.30503373, 0.51972874],
       [0.16392141, 0.66666667, 0.83333333, 0.44032988, 0.66568158,
        0.38604651, 0.49465378],
       [0.16411987, 0.66666667, 0.83333333, 0.50992369, 0.633003  ,
        0.33140473, 0.49447151],
       [0.16431832, 0.66666667, 0.83333333, 0.50761729, 0.61803431,
        0.62141008, 0.57884557],
       [0.16451677, 0.66666667, 0.83333333, 0.46913516, 0.46799589,
        0.43305629, 0.46683875],
       [0.16471522, 0.66666667, 0.83333333, 0.43184296, 0.71846233,
        0.36597951, 0.5033579 ],
       [0.16491367, 0.66666667, 0.83333333, 0.37770263, 0.72043274,
        0.44064234, 0.47744734],
       [0.16511213, 0.66666667, 0.83333333, 0.58917584, 0.61756397,
        0.38988345, 0.29738863],
       [0.16531058, 0.66666667, 0.83333333, 0.66306578, 0.54698481,
        0.29816112, 0.44046069],
       [0.16550903, 0.66666667, 0.83333333, 0.34753011, 0.62429571,
        0.42018128, 0.54422186],
       [0.16570748, 0.66666667, 0.83333333, 0.42084169, 0.59899732,
        0.34538273, 0.68062549],
       [0.16590593, 0.66666667, 0.83333333, 0.53714766, 0.5233176 ,
        0.31867285, 0.51455606],
       [0.16610439, 0.66666667, 0.83333333, 0.46459716, 0.79728447,
        0.29287657, 0.54589608],
       [0.16630284, 0.66666667, 0.83333333, 0.60685885, 0.7627859 ,
        0.53102514, 0.36463042],
       [0.16650129, 0.66666667, 0.83333333, 0.45016676, 0.5371357 ,
        0.36234758, 0.48067005],
       [0.16669974, 0.66666667, 0.83333333, 0.32630458, 0.6502717 ,
        0.29977745, 0.31127834],
       [0.16689819, 0.66666667, 0.83333333, 0.61934091, 0.68324217,
        0.43857282, 0.40336865],
       [0.16709665, 0.66666667, 0.83333333, 0.53289786, 0.64045468,
        0.4707726 , 0.33293981],
       [0.1672951 , 0.66666667, 0.83333333, 0.62172647, 0.80239406,
        0.24065814, 0.45971052],
       [0.16749355, 0.66666667, 0.83333333, 0.56874513, 0.55327686,
        0.30763078, 0.42705133],
       [0.167692  , 0.66666667, 0.83333333, 0.75409067, 0.70157827,
        0.33767338, 0.40823682],
       [0.16789045, 0.66666667, 0.83333333, 0.55806708, 0.59841169,
        0.42723556, 0.46425731],
       [0.16808891, 0.66666667, 0.83333333, 0.69033112, 0.650277  ,
        0.57144895, 0.3518286 ],
       [0.16828736, 0.66666667, 0.83333333, 0.53220468, 0.77556955,
        0.54382698, 0.49633586],
       [0.16848581, 0.66666667, 0.83333333, 0.52509087, 0.41871796,
        0.53415646, 0.3309969 ],
       [0.16868426, 0.66666667, 0.83333333, 0.6630777 , 0.36348843,
        0.21869866, 0.42808497],
       [0.16888271, 0.66666667, 0.83333333, 0.66012703, 0.50377388,
        0.46911669, 0.42108926],
       [0.16908117, 0.66666667, 0.83333333, 0.65719551, 0.36174508,
        0.30243667, 0.66768381],
       [0.16927962, 0.66666667, 0.83333333, 0.62825936, 0.57472729,
        0.38454321, 0.64870015],
       [0.16947807, 0.66666667, 0.83333333, 0.61402683, 0.81333739,
        0.35459519, 0.65223118],
       [0.16967652, 0.66666667, 0.83333333, 0.64476372, 0.61324651,
        0.3916903 , 0.3364507 ],
       [0.16987498, 0.66666667, 0.83333333, 0.71781102, 0.65292058,
        0.24742094, 0.57611908],
       [0.17007343, 0.66666667, 0.83333333, 0.5376592 , 0.78369732,
        0.42405026, 0.6207278 ],
       [0.17027188, 0.66666667, 0.83333333, 0.58864504, 0.65972945,
        0.36872454, 0.51423921],
       [0.17047033, 0.66666667, 0.83333333, 0.45362191, 0.68462962,
        0.53711181, 0.36674166],
       [0.17066878, 0.66666667, 0.83333333, 0.53549992, 0.49845023,
        0.21808856, 0.4229779 ],
       [0.17086724, 0.66666667, 0.83333333, 0.56127937, 0.67585306,
        0.29947504, 0.35929855],
       [0.17106569, 0.66666667, 0.83333333, 0.58591761, 0.61611436,
        0.35467961, 0.45090625],
       [0.17126414, 0.66666667, 0.83333333, 0.64491204, 0.69148971,
        0.20742524, 0.48409945],
       [0.17146259, 0.66666667, 0.83333333, 0.85195323, 0.62527227,
        0.35117042, 0.25665912],
       [0.17166104, 0.66666667, 0.83333333, 0.57876465, 0.5611623 ,
        0.4816895 , 0.37922238],
       [0.1718595 , 0.66666667, 0.83333333, 0.48242571, 0.56956496,
        0.29905694, 0.29823658],
       [0.17205795, 0.66666667, 0.83333333, 0.54745425, 0.61160209,
        0.27894404, 0.48696811],
       [0.1722564 , 0.66666667, 0.83333333, 0.62351399, 0.6417171 ,
        0.51692447, 0.48082924],
       [0.17245485, 0.66666667, 0.83333333, 0.67285432, 0.76966338,
        0.44123265, 0.44093442],
       [0.1726533 , 0.66666667, 0.83333333, 0.49849421, 0.54406418,
        0.32597676, 0.5700005 ],
       [0.17285176, 0.66666667, 0.83333333, 0.8202298 , 0.46685339,
        0.51506072, 0.46946898],
       [0.17305021, 0.66666667, 0.83333333, 0.728321  , 0.69478484,
        0.49997131, 0.4046414 ],
       [0.17324866, 0.66666667, 0.83333333, 0.58467526, 0.88585612,
        0.3456766 , 0.40502593],
       [0.17344711, 0.66666667, 0.83333333, 0.56592784, 0.55894564,
        0.6284823 , 0.50010475],
       [0.17364556, 0.66666667, 0.83333333, 0.61675033, 0.72906911,
        0.35640598, 0.35105949],
       [0.17384402, 0.66666667, 0.83333333, 0.75741633, 0.60916402,
        0.55898595, 0.60063631],
       [0.17404247, 0.66666667, 0.83333333, 0.67919303, 0.35581607,
        0.19058489, 0.35706113],
       [0.17424092, 0.66666667, 0.83333333, 0.73898873, 0.63057041,
        0.50616972, 0.50462219],
       [0.17443937, 0.66666667, 0.83333333, 0.55369508, 0.65903964,
        0.4542438 , 0.37052034],
       [0.17463782, 0.66666667, 0.83333333, 0.53888605, 0.32299718,
        0.49203166, 0.33765294],
       [0.17483628, 0.66666667, 0.83333333, 0.52500914, 0.5056947 ,
        0.2946801 , 0.42697432],
       [0.17503473, 0.66666667, 0.83333333, 0.76213175, 0.57095799,
        0.25609406, 0.57514827],
       [0.17523318, 0.66666667, 0.83333333, 0.77096287, 0.49835092,
        0.22879404, 0.39076878],
       [0.17543163, 0.66666667, 0.83333333, 0.70702404, 0.56699757,
        0.28395286, 0.4250315 ],
       [0.17563009, 0.66666667, 0.83333333, 0.80159978, 0.6493734 ,
        0.60192788, 0.41643542],
       [0.17582854, 0.66666667, 0.83333333, 0.69704148, 0.58654211,
        0.15221453, 0.25923464],
       [0.17602699, 0.66666667, 0.83333333, 0.51044309, 0.58028864,
        0.18375871, 0.3012168 ],
       [0.17622544, 0.66666667, 0.83333333, 0.58040526, 0.89435979,
        0.43123124, 0.44607091]]]).astype(np.float32)
    try:
        prediction = model.predict(seq_array)
        
        pred = prediction.tolist()
        return(pred)
    except Exception as e:
        return(str(e))
    
if __name__ == "__main__":
    init()
    #main()
    run("{\"score_df\": [{\"s20\": 0.5581395348837184, \"s10\": 0.0, \"s2\": 0.5451807228915584, \"s21\": 0.6618337475835432, \"s9\": 0.12761374854168395, \"s19\": 0.0, \"s3\": 0.31066056245912677, \"cycle\": 1, \"s15\": 0.3089649865332831, \"s4\": 0.2694125590817009, \"s1\": 0.0, \"s11\": 0.2083333333333357, \"cycle_norm\": 0.0, \"s13\": 0.2205882352941444, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.2121212121210192, \"s14\": 0.1321601816492901, \"s6\": 1.0, \"setting2\": 0.75, \"setting1\": 0.632183908045977, \"s12\": 0.6460554371002161, \"s17\": 0.3333333333333357, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.6521739130434696}, {\"s20\": 0.6821705426356601, \"s10\": 0.0, \"s2\": 0.15060240963856586, \"s21\": 0.6868268434134208, \"s9\": 0.14668401687158195, \"s19\": 0.0, \"s3\": 0.37955090473076325, \"cycle\": 2, \"s15\": 0.21315890727203168, \"s4\": 0.2223160027008788, \"s1\": 0.0, \"s11\": 0.38690476190476275, \"cycle_norm\": 0.002770083102493075, \"s13\": 0.26470588235270043, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.16666666666696983, \"s14\": 0.20476829394158358, \"s6\": 1.0, \"setting2\": 0.25, \"setting1\": 0.3448275862068965, \"s12\": 0.7398720682302695, \"s17\": 0.4166666666666714, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.8051529790660226}, {\"s20\": 0.7286821705426334, \"s10\": 0.0, \"s2\": 0.3765060240963862, \"s21\": 0.7213476940071786, \"s9\": 0.15808130664991182, \"s19\": 0.0, \"s3\": 0.34663178548071016, \"cycle\": 3, \"s15\": 0.4586379376683354, \"s4\": 0.3222484807562438, \"s1\": 0.0, \"s11\": 0.38690476190476275, \"cycle_norm\": 0.0055401662049861505, \"s13\": 0.2205882352941444, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.22727272727297532, \"s14\": 0.15564041696769948, \"s6\": 1.0, \"setting2\": 0.5833333333333334, \"setting1\": 0.5172413793103449, \"s12\": 0.6993603411513902, \"s17\": 0.4166666666666714, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.6859903381642596}]}")

In [2]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.38.0 to work with aml_bm_00


# Deploy the model as a web service

You have registered a machine learning model that predicts errors. To support this scenario, you will deploy the model as a web service.

First, let's determine what models you have registered in the workspace.

In [3]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

machineRULPredictor_model version: 1




Right, now let's get the model that we want to deploy. By default, if we specify a model name, the latest version will be returned.

In [ ]:
model = ws.models['machineRULPredictor_model']
print(model.name, 'version', model.version)

We're going to create a web service to host this model, and this will require some code and configuration files; so let's create a folder for those.

In [4]:
# Set path for scoring script and environment files
script_file = os.path.join("./config/lstmscore.py")
env_file = os.path.join("./config/webservices_conda.yaml")

The web service where we deploy the model will need some Python code to load the input data, get the model from the workspace, and generate and return predictions. We'll save this code in an *entry script* (often called a *scoring script*) that will be deployed to the web service:

In [ ]:
#%%writefile $script_file
#import json
# import joblib
#import tensorflow as tf
#import numpy as np
#from azureml.core.model import Model

# Called when the service is loaded
#def init():
#    global model
    # Get the path to the deployed model file and load it
#    model_path = Model.get_model_path('machine1ErrorPredictor_model')
    # model = joblib.load(model_path)
#    model = tf.keras.models.load_model(model_path)

# Called when a request is received
#def run(raw_data):
    # Get the input data as a numpy array
#    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
#    predictions = model.predict(data)
    # Return the predictions as JSON
#    return json.dumps(predictions.tolist())

The web service will be hosted in a container, and the container will need to install any required Python dependencies when it gets initialized. In this case, our scoring code requires the **tensorflow** and **azureml-defaults** packages, so we'll create a .yml file that tells the container host to install them into the environment.

In [ ]:
#%%writefile $env_file
#name: inference_env
#dependencies:
#- python=3.6.2
#- tensorflow=2.0.0
#- pip
#- pip:
#  - azureml-defaults
#  - azure
#  - azure-ml-api-sdk
#  - tensorflow

Now you're ready to deploy. We'll deploy the container a service named **machinerulpredictor-service**. The deployment process includes the following steps:

1. Define an inference configuration, which includes the scoring and environment files required to load and use the model.
2. Define a deployment configuration that defines the execution environment in which the service will be hosted. In this case, an Azure Container Instance.
3. Deploy the model as a web service.
4. Verify the status of the deployed service.

> **More Information**: For more details about model deployment, and options for target execution environments, see the [documentation](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where).

Deployment will take some time as it first runs a process to create a container image, and then runs a process to create a web service based on the image. When deployment has completed successfully, you'll see a status of **Healthy**.

In [ ]:
# from azureml.core.webservice import Webservice
service=Webservice(ws, "machinerulpredictor-service")
#service.delete()

In [5]:
# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "machinerulpredictor-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-03-04 22:20:49+00:00 Creating Container Registry if not exists.
2022-03-04 22:20:50+00:00 Registering the environment.
2022-03-04 22:20:52+00:00 Use the existing image.
2022-03-04 22:20:52+00:00 Generating deployment configuration.
2022-03-04 22:20:54+00:00 Submitting deployment to compute.
2022-03-04 22:20:57+00:00 Checking the status of deployment machinerulpredictor-service..
2022-03-04 22:23:11+00:00 Checking the status of inference endpoint machinerulpredictor-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


Hopefully, the deployment has been successful and you can see a status of **Healthy**. If not, you can use the following code to get the service logs to help you troubleshoot.

In [ ]:
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
# service.delete()

Take a look at your workspace in [Azure Machine Learning Studio](https://ml.azure.com) and view the **Endpoints** page, which shows the deployed services in your workspace.

You can also retrieve the names of web services in your workspace by running the following code:

In [8]:
for webservice_name in ws.webservices:
    print(webservice_name)
    print(service.scoring_uri)
    # print(service.swagger_uri)

machinerulpredictor-service
http://1561ac3c-52d7-4948-8625-e7fbfbd9d6b4.eastus.azurecontainer.io/score


## Use the web service

With the service deployed, now you can consume it from a client application.

In [9]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-consume-web-service?tabs=azure-cli

import urllib
import json 
import requests
import pandas as pd

# The URL will need to be editted after service create.
url = 'http://1561ac3c-52d7-4948-8625-e7fbfbd9d6b4.eastus.azurecontainer.io/score'

## Sequence length will need to match the training sequence length from
## 2_model_building_and_evaluation.ipynb
sequence_length = 137

# We'll read in this data to test the service
test_df = pd.read_csv('./data/preparedData/PM_test_prepared.csv', sep=",", header=0)

# Labels are what we're predicting.
label_cols = ['label1', 'label2', 'RUL']

# The scoreing data should not have labels... if we knew the label, 
# we wouldn't need to predict.
score_df = test_df.drop(label_cols, axis = 1)
headers = {'Content-Type':'application/json'}

# Now get the machine numbers, for each machine get the 
# prediction for the label timepoint
machineID = score_df['machineID'].unique()

for ind in machineID:
    
    try:
        body = score_df[score_df.machineID==ind]
        print('ID {}: size {}'.format(ind, body.shape))
        #if body.shape[0] < sequence_length : 
        #    print("Skipping machineID {} as we need {} records to score and only have {} records.".format(ind, sequence_length, body.shape[0]))
        #    continue
        print('ID {}: {} \t {}'.format(ind, body.shape, body.tail(sequence_length+ 10).shape))
        body = "{\"score_input\": " + \
                    body.tail(sequence_length+10).to_json(orient="records") +\
                    "}"
        req = urllib.request.Request(url, str.encode(body), headers)
        with urllib.request.urlopen(req) as response:
            the_page = response.read()
            print('ID {}: {}'.format(ind,the_page))
        
    except urllib.error.HTTPError as error:
        print("The request failed with status code {}: \n{}".format(error, error.read))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.reason)      

ID 89: size (137, 9)
ID 89: (137, 9) 	 (137, 9)
ID 89: b'[[0.19713306427001953]]'


In [ ]:
print(service.get_logs())

In [ ]:
service.delete()
print ('Service deleted.')

For more information about publishing a model as a service, see the [documentation](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)


# Conclusion

Working through all of these notebooks, we have completed:

 * Data aquisition ingestion and preparation in `Code/1_data_ingestion_and_preparation.ipynb` notebook.
 * Building an LSTM time series model that used a 50 cycle history from an aircraft engine to predict failure within the next 30 cycles in the `Code/2_model_building_and_evaluation.ipynb` notebook.
 * Operationalization asset generation and model deployment in the `Code/3_operationalization.ipynb` notebook.
 
This scenario is intended to help guide you through the predictive maintenance model development process with your own data. 

    
 